In [1]:
# load packages
import ixmp                    # package for ix modeling platform
import message_ix              # package for MESSAGEix model
from message_ix.utils import make_df

<IPython.core.display.Javascript object>

In [2]:
# Loading the platform (and getting conected to a local database)
mp = ixmp.Platform()

In [3]:
# create empty scenario
baseline = message_ix.Scenario(mp, model='MESSAGEix South Africa', 
                               scenario='baseline', version='new')


In [4]:
# read South Africa model from Excel (init_items = True is important as this scenario also includes MACRO)
baseline.read_excel("../resources/MESSAGEix_South_Africa.xlsx", add_units=True, commit_steps=False, init_items=True)

In [5]:
#baseline.remove_solution()
baseline.check_out()
baseline.add_set('commodity', 'hydrogen')


In [6]:
mp.scenario_list(model='MESSAGEix South Africa')

,model,scenario,scheme,is_default,is_locked,cre_user,cre_date,upd_user,upd_date,lock_user,lock_date,annotation,version
0,MESSAGEix South Africa,added fuel,MESSAGE,1,0,jankle,2022-02-23 13:11:46.569000,jankle,2022-02-23 13:53:10.317000,None,None,clone Scenario from 'MESSAGEix South Africa|ba...,1
1,MESSAGEix South Africa,added fuels,MESSAGE,1,1,jankle,2022-02-23 11:52:24.386000,None,None,jankle,2022-02-23 11:55:09.099000,clone Scenario from 'MESSAGEix South Africa|ba...,1
2,MESSAGEix South Africa,emission budget,MESSAGE,1,0,jankle,2022-02-16 14:50:54.405000,jankle,2022-02-16 15:49:37.105000,None,None,clone Scenario from 'MESSAGEix South Africa|ba...,1
3,MESSAGEix South Africa,extended fuel supply,MESSAGE,1,1,jankle,2022-02-23 08:45:47.932000,None,None,jankle,2022-02-23 08:50:14.484000,clone Scenario from 'MESSAGEix South Africa|ba...,1
4,MESSAGEix South Africa,extended with h2 and nh3,MESSAGE,1,1,jankle,2022-02-23 08:53:19.216000,jankle,2022-02-23 09:10:11.633000,jankle,2022-02-23 11:38:00.511000,clone Scenario from 'MESSAGEix South Africa|ba...,1
5,MESSAGEix South Africa,full hydrogen,MESSAGE,1,1,jankle,2022-02-18 16:05:13.772000,jankle,2022-02-22 15:33:58.216000,jankle,2022-02-22 16:12:20.071000,clone Scenario from 'MESSAGEix South Africa|ba...,1
6,MESSAGEix South Africa,hydrogen use,MESSAGE,1,0,jankle,2022-02-17 12:59:52.812000,jankle,2022-02-17 14:28:01.984000,None,None,clone Scenario from 'MESSAGEix South Africa|ba...,1


In [8]:
baseline.commit('init')
baseline.solve()

In [9]:
# check objective function value
baseline.var('OBJ')

{'lvl': 63014.5703125, 'mrg': 0.0}

In [10]:
# check emissions
baseline.var('EMISS')

,node,emission,type_tec,year,lvl,mrg
0,World,CO2,all,2020,514.450798,0.0
1,World,CO2,all,2030,648.868289,0.0
2,World,CO2,all,2040,733.833429,0.0
3,World,CO2,all,2050,866.594934,0.0
4,World,CO2,all,2060,1028.482914,0.0
5,World,CO2,all,2070,1256.080001,0.0
6,World,CH4,all,2020,13.374521,0.0
7,World,CH4,all,2030,15.464832,0.0
8,World,CH4,all,2040,16.595169,0.0
9,World,CH4,all,2050,19.105427,0.0


In [34]:
# clone baseline scenario for modification
scenario = baseline.clone('MESSAGEix South Africa', 'hydrogen use','copy of the baseline scenario with added hydrogen',
                  keep_solution=False)

In [40]:
# check out scenario for editing
scenario.check_out()

In [65]:
scenario.cat_list('node')

['economy']

In [61]:
# add hydrogen as commodity
country = 'South Africa'
# Adding required information via MESSAGEix sets
commodity= 'hydrogen_fuel'
scenario.add_set('commodity', commodity)
years = [2030, 2040, 2050, 2060, 2070]
demand_vector = [4.5, 5.00, 6.00, 7.00, 8.00]

# Building the required table (python DataFrame)
import pandas as pd
df = pd.DataFrame({'node': country,
                   'commodity': commodity,
                   'level': 'useful',
                   'year': years,
                   'value': demand_vector,    # cup of coffee per year
                   'unit': 'GWa/a',
                   'time': 'year',
                   'mode':'all',
                   })
scenario.add_par('demand', df)


In [87]:
scenario.set('commodity')

0          methanol
1          rc_therm
2              coal
3           i_therm
4           rc_spec
5         transport
6            i_feed
7           biomass
8          non-comm
9            electr
10         lightoil
11         crudeoil
12              gas
13          fueloil
14           i_spec
15             wind
16         solar_th
17            hydro
18         shalegas
19         solar_pv
20    hydrogen_fuel
dtype: object

In [90]:
# add supply technology for hydrogen 
# scenario.par('input')
# scenario.par('output', filters={'year_act':2050, 'level':'useful'})
tecs = ['h2_smr','h2_pem']
scenario.add_set('technology',tecs)
costs = {'h2_smr':1,'h2_pem':2} # cost of water, electricity, coffee beans
base_df = pd.DataFrame({'node_loc':country,'year_vtg':years,'year_act':years,'mode':'M1','time':'year','unit':'USD/kWa'})
for tec, value in costs.items():
    df = make_df(base_df,technology=tec,value=value)
    scenario.add_par('var_cost',df)
scenario.par('var_cost', filters={'technology':'h2_msr'})
# add costs for hydrogen
# add demand for hydrogen

,node_loc,technology,year_vtg,year_act,mode,time,value,unit
0,South Africa,h2_msr,2030,2030,M1,year,1.0,USD/kWa
1,South Africa,h2_msr,2040,2040,M1,year,1.0,USD/kWa
2,South Africa,h2_msr,2050,2050,M1,year,1.0,USD/kWa
3,South Africa,h2_msr,2060,2060,M1,year,1.0,USD/kWa
4,South Africa,h2_msr,2070,2070,M1,year,1.0,USD/kWa


In [93]:
scenario.remove_set('technology', 'h2_msr')

In [99]:
# electricity input to h2_pem

df = pd.DataFrame({'node_loc':country, 'technology':'h2_pem','node_origin':country,'mode':'M1',
                   'commodity': 'electr','year_vtg':years,'year_act': years,
                   'level':'final','time':'year','time_origin':'year','value':1.4,'unit':'kWa'})

scenario.add_par('input',df)

#  gas to h2_smr

df = pd.DataFrame({'node_loc':country, 'technology':'h2_smr','node_origin':country,'mode':'M1',
                   'commodity': 'gas','year_vtg':years,'year_act': years,
                   'level':'final','time':'year','time_origin':'year','value':2,'unit':'kWa'})

scenario.add_par('input',df)

In [100]:
# Adding data to the parameter "output" for the technology
df = pd.DataFrame({'node_loc': country,
                   'node_dest': country,
                   'technology': 'h2_pem',
                   'commodity': 'hydrogen_fuel',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 1,
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
scenario.add_par('output', df)
# Adding data to the parameter "output" for the technology
df = pd.DataFrame({'node_loc': country,
                   'node_dest': country,
                   'technology': 'coffee_maker',
                   'commodity': 'coffee_cup',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 1,
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
scenario.add_par('output', df)

,node_loc,technology,year_vtg,year_act,mode,node_origin,commodity,level,time,time_origin,value,unit
0,South Africa,bio_extr,2010,2010,M1,South Africa,biomass,renewable,year,year,1.0,-
1,South Africa,bio_extr,2020,2020,M1,South Africa,biomass,renewable,year,year,1.0,-
2,South Africa,bio_extr,2030,2030,M1,South Africa,biomass,renewable,year,year,1.0,-
3,South Africa,bio_extr,2040,2040,M1,South Africa,biomass,renewable,year,year,1.0,-
4,South Africa,bio_extr,2050,2050,M1,South Africa,biomass,renewable,year,year,1.0,-
...,...,...,...,...,...,...,...,...,...,...,...,...
3269,South Africa,h2_smr,2030,2030,M1,South Africa,gas,final,year,year,2.0,kWa
3270,South Africa,h2_smr,2040,2040,M1,South Africa,gas,final,year,year,2.0,kWa
3271,South Africa,h2_smr,2050,2050,M1,South Africa,gas,final,year,year,2.0,kWa
3272,South Africa,h2_smr,2060,2060,M1,South Africa,gas,final,year,year,2.0,kWa


In [ ]:
# add cumulative emission constraint for CO2
scenario.add_par('bound_emission', ['South Africa', 'CO2', 'all', 'cumulative'],
             value=600., unit='MtCO2')

In [ ]:
# list units defined on platform
mp.units()

In [ ]:
# add MtCO2 as unit to platform
mp.add_unit("MtCO2")

In [38]:
# commit scenario to platform
scenario.commit("added hydrogen demand")

In [39]:
# solve scenario with CO2 emission budget
scenario.solve()

ModelError: GAMS errored with return code 3:
    There was an execution error

For details, see the terminal output above, plus:
Listing   : C:\Users\jankle\Anaconda3\envs\mesenv\lib\site-packages\message_ix\model\MESSAGE_run.lst
Input data: C:\Users\jankle\Anaconda3\envs\mesenv\lib\site-packages\message_ix\model\data\MsgData_MESSAGEix_South_Africa_hydrogen_use.gdx

In [ ]:
# check objective function value
scenario.var('OBJ')

In [ ]:
# check emissions
scenario.var('EMISS')

In [27]:
# close platform connection
mp.close_db()